In [44]:
import pandas as pd
import numpy as np
import os

# read all files
folderpath = r"F:\BOCIF\7688"
filepaths  = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]
all_files = []

for path in filepaths:
    with open(path,errors='ignore', mode='r') as f:
        file = f.readlines()
        all_files.append(file)

In [245]:
# extract date, commission, pledge amount, margin occupied, fund avail, 特殊贡献返还
list_all_date = []
list_all_fund_avail = []
list_commission =[]
list_date = []
list_特殊贡献返还 = []
list_pledge_amount = []
list_margin_occupied =[]
list_fund_avail = []

for file in all_files:
    date =  file[1].split()[2][5:]
    list_all_date.append(date)
    
    all_fund_avail = file[19].split()[-1]
    list_all_fund_avail.append(all_fund_avail)
    
    commission = file[14].split()[4]
    list_commission.append(commission)
    
    for line in file:
        if '特殊贡献返还' in line:
            
            date = line.split('|')[1]
            list_date.append(date)
            
            特殊贡献返还 = float(line.split('|')[3])
            list_特殊贡献返还.append(特殊贡献返还)
            
            pledge_amount = float(file[11].split()[-1])
            list_pledge_amount.append(pledge_amount)
            
            margin_occupied = float(file[14].split()[-1])
            list_margin_occupied.append(margin_occupied)
            
            fund_avail = float(file[19].split()[-1])
            list_fund_avail.append(fund_avail)
        
        if '特殊贡献返还-利息' in line:
            print(line)

|20210625|出入金                  |       116023.63|            0.00|特殊贡献返还-利息                         |

|20210928|出入金                  |       449789.11|            0.00|特殊贡献返还-利息                         |

|20211227|出入金                  |       618695.39|            0.00|特殊贡献返还-利息                         |

|20220325|出入金                  |       525575.52|            0.00|特殊贡献返还-利息                         |

|20220627|出入金                  |       551240.29|            0.00|特殊贡献返还-利息                         |

|20220927|出入金                  |       107098.17|            0.00|特殊贡献返还-利息                         |



In [246]:
df = pd.DataFrame(index=list_all_date)
df['手续费'] = list_commission
df['可用资金'] = list_all_fund_avail
df = df.truncate(after='20221123') # cannot read files after 20221123
df = df.astype({'手续费': 'float','可用资金': 'float'})
df.index = pd.to_datetime(df.index)
df['year'] = df.index.year
df['month'] = df.index.month
df = df.groupby(['year','month']).sum()
df = df.rename(columns={'手续费':'月手续费','可用资金':'月可用资金'})

In [247]:
list_ratio = []
for i in range(len(list_pledge_amount)):
    if list_pledge_amount[i]>list_margin_occupied[i]:  # compare pledge amount with margin occupied
        list_fund_avail[i] = round(list_fund_avail[i] - (list_pledge_amount[i]-list_margin_occupied[i]),2) # arrive at new fund avail
    list_ratio.append(list_特殊贡献返还[i]/list_fund_avail[i]) # calculate 特殊贡献返还/fund avail

In [249]:
results = pd.DataFrame(index=list_date)
results['特殊贡献返还'] = list_特殊贡献返还
results['可用资金'] = list_fund_avail
results['特殊贡献返还/可用资金'] = list_ratio
results = results.drop(index=['20210625','20210928','20211227','20220325','20220627','20220927']) # exclude 特殊贡献返还-利息
results.index = pd.to_datetime(results.index)
results['year'] = results.index.year
results['month'] = results.index.month
results.index.name = '日期'
results = pd.merge(results,df,left_on=['year','month'],right_index=True)
results = results.drop(columns = ['year','month'])
results

,特殊贡献返还,可用资金,特殊贡献返还/可用资金,月手续费,月可用资金
日期,,,,,
2021-06-23,18795.53,6.164480e+07,0.000305,53603.71,9.837577e+08
2021-07-20,9427.99,3.843924e+07,0.000245,77957.23,1.367535e+09
2021-08-20,13711.50,9.844165e+07,0.000139,62620.51,2.590949e+09
2021-09-22,11014.73,1.683653e+08,0.000065,0.00,3.368513e+09
2021-11-23,3557.73,1.005349e+08,0.000035,62714.81,2.846581e+09
2021-12-30,8271.74,1.641159e+08,0.000050,14159.79,3.396060e+09
2022-01-27,4980.83,5.762759e+07,0.000086,6069.27,2.278192e+09
2022-06-24,68983.25,4.034610e+07,0.001710,20242.98,1.330615e+09
2022-07-27,4802.95,9.672374e+06,0.000497,15181.59,1.161757e+09


In [243]:
results.to_excel('F:\BOCIF\提取数据.xlsx')